# Load library

In [ ]:
#Import lib
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from my_useful_functions import calculate_performance_statistical_parity,calculate_performance_equalized_odds,calculate_performance_equal_opportunity,calculate_performance_predictive_parity,calculate_performance_predictive_equality,calculate_performance_treatment_equality
from sklearn import preprocessing
from aif360.datasets.binary_label_dataset import BinaryLabelDataset
# DT
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
# NB
from sklearn.naive_bayes import GaussianNB
# MLP
from sklearn.neural_network import MLPClassifier
# kNN
from sklearn.neighbors import KNeighborsClassifier
# Agarwal
from exponentiated_gradient_reduction import ExponentiatedGradientReduction
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
# DIR
from disparate_impact_remover import DisparateImpactRemover
# LFR
from learning_fair_representations import LFR
# EOP
from eq_odds_postprocessing import EqOddsPostprocessing
# CEP
from calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing
from compute_abroca import *
import numpy as np
import sklearn.metrics as metrics 
from scipy import interpolate
from scipy import integrate
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl

from sklearn import metrics
import seaborn as sns
import seaborn as sn


import seaborn; seaborn.set_style('whitegrid')
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import OrdinalEncoder

#from aif360.datasets import StandardDataset
#from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

%matplotlib inline

from pomegranate import *
import pygraphviz
numpy.random.seed(0)
numpy.set_printoptions(suppress=True)
from sklearn.model_selection import train_test_split
import networkx
from pomegranate.utils import plot_networkx
#matplotlib.use('TkAgg')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Data analysis

## Statistic Information

In [ ]:
df=pd.read_csv("data/german_data_credit.csv")
risk={1:"Good", 2:"Bad"}
df["class-label"]=df["class-label"].map(risk)

In [ ]:
df.info()

In [ ]:
#statistical_parity
def statistical_parity_difference(dataset, protected,classes,majority_group,minority_group,positive_value):
    ratio = round(pd.crosstab(dataset[protected], dataset[classes]).div(pd.crosstab(dataset[protected], dataset[classes]).apply(sum,1),0),4)*100
    #return ratio
    return ratio.loc[majority_group][positive_value]-ratio.loc[minority_group][positive_value]

In [ ]:
def basic_info(dataset,class_label,positive_value,negative_value,protected_attr,majority_group,minority_group):
    categorical_columns = (dataset.select_dtypes(['category', 'object']).columns)
    numeric_columns = (dataset.select_dtypes(['number']).columns)
    for attr in dataset.keys():
        if attr in categorical_columns:    
            print(attr ,' & Categorical & ',len(dataset[attr].value_counts()),'& &',dataset[attr].isnull().sum(),'\\\\')
        else:           
            print(attr,'& Numerical &',len(dataset[attr].value_counts()),'&',min(dataset[attr]),"-",max(dataset[attr]),'&',dataset[attr].isnull().sum(),'\\\\')  
    
    print('Class distribution:',dataset[class_label].value_counts())
    print('Class imbalance:',len(dataset.loc[dataset[class_label]==negative_value])/len(dataset.loc[dataset[class_label]==positive_value]))
    print('statistical parity:',statistical_parity_difference(dataset,protected_attr,class_label,majority_group,minority_group,positive_value))

In [ ]:
basic_info(df,class_label="class-label",positive_value="Good",negative_value="Bad",
           protected_attr="sex",majority_group='male',minority_group='female')

## Missing value

In [ ]:
#Check missing values
print('Missing values')
for i,j in zip(df.columns,(df.values.astype(str) == '?').sum(axis = 0)):
    if j > 0:
        print(str(i) + ': ' + str(j) + ' records')
print("---------------------------------")
df.isnull().sum()

In [ ]:
df.info()

## Bayesian Network

In [ ]:
def plot_bayesian_network(a, X, labels, name):
  model = BayesianNetwork.from_samples(X, algorithm='exact',constraint_graph=a,state_names=labels)
  plt.figure(figsize=(15, 12),dpi=400)
  model.plot()
  print(model.structure)
  plt.savefig(name, bbox_inches='tight')

In [ ]:
def german_credit():
  df=pd.read_csv("data/german_data_credit.csv")
  print("Length:",len(df))
  print("Number of attribute:",len(df.columns))
  
  print(df['class-label'].value_counts())

  X = df.values
  for i in range(len(list(df.keys()))):
    if len(pd.unique(df[df.keys()[i]])) == 2:
      print(i,list(df.keys())[i], len(pd.unique(df[df.keys()[i]]))) 
  labels=list(df.keys())

  X[:,1]=np.array([0 if v<=6 else 1 if v>6 and v<=12 else 2 for v in X[:,1]]) #duration
  X[:,4]=np.array([0 if v<=2000 else 1 if v>2000 and v<=5000 else 2 for v in X[:,4]]) #credit amount
  X[:,11]=np.array([0 if v<25 else 1 for v in X[:,11]]) #age
 
  a = networkx.DiGraph()
  b = tuple((0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20))
  c= tuple((21, ))
  a.add_edge(b,c)
  a.add_edge(b,b)

  return a, X, labels

In [ ]:
a, X, labels = german_credit()
plot_bayesian_network(a, X, labels, 'German.BN.pdf')

In [ ]:
def german_credit_plot_1():
  df=pd.read_csv("data/german_data_credit.csv")
  print("Length:",len(df))
  print("Number of attribute:",len(df.columns))

  print(len(df[(df['credit-amount'] < 5000) & (df['credit-amount'] >= 2000) & (df["duration"] > 48)]))

  a24 = [422, 284, 64]
  a48 = [10, 96, 108]
  a72 = [108, 0, 16]

  
  groups = ("<= 2000", "2000 - 5000", "> 5000")
  groups_value = {
      '0-2 years': (422, 284, 64),
      '2-4 years': (10, 96, 108),
      '4-6 years': (108, 0, 16),
  }

  x = np.arange(len(groups))  # the label locations
  width = 0.2  # the width of the bars
  multiplier = 0
  

  fig, ax = plt.subplots(layout='constrained', figsize=(12, 6))

  column_sums = np.sum(list(groups_value.values()), axis=0)
  for attribute, measurement in groups_value.items():
      offset = width * multiplier
      percentage = [100 * value / column_sums[index] for index, value in enumerate(measurement)]
      # rects = ax.bar(x + offset, [round(_, 2) for _ in percentage], width, label=attribute)
      rects = ax.bar(x + offset, measurement, width, label=attribute)
      ax.bar_label(rects, labels=[f"{(round(_, 2))}%" for _ in percentage], padding=4)
      multiplier += 1

  # Add some text for labels, title and custom x-axis tick labels, etc.
  ax.set_ylabel('Population', fontsize=14, fontweight='bold')
  ax.set_xlabel('Credit amount', fontsize=14, fontweight='bold')
  # ax.set_title('Penguin attributes by species')
  ax.set_xticks(x + width, groups)
  plt.legend(title="Duration", loc='upper left', bbox_to_anchor=(0.3, 1))

  plt.show()

In [ ]:
german_credit_plot_1()

In [ ]:
import matplotlib.ticker as mtick
def plot_2_variable(dataset,var1, var2, width=0.5, wide=9,height=5,legend="", xlabel="", logy=False,verbose=True,indexlist="",index=False):
    
    var1_var2_ratio=round(pd.crosstab(dataset[var1], dataset[var2]).div(pd.crosstab(dataset[var1], dataset[var2]).apply(sum,1),0),3)*100
    var1_var2 =  pd.crosstab(dataset[var1], dataset[var2])
    if verbose:
        print(var1_var2)
        
    if index:
        var1_var2 = var1_var2.reindex(indexlist)
        var1_var2_ratio = var1_var2_ratio.reindex(indexlist)
    
    ax = var1_var2.plot(kind='bar',width=width,figsize=(wide,height)
                     ,logy=logy
                     )
    #x = hour_income_gender.plot(kind ='bar',stacked=False)
    xlocs, xlabs = plt.xticks()
    for i in range(len(var1_var2)):
        for j in range(len(var1_var2.iloc[i])):
            plt.text(xlocs[i]-0.35+j*0.3, var1_var2.iloc[i][j] + 2,str(round(var1_var2_ratio.iloc[i][j],1))+"%" )
    #plt.title('Distribution of '+ title+ ' across '+ legend)
    plt.ylabel('Population',fontsize=14,fontweight='bold')
    plt.xlabel(xlabel,fontsize=14,fontweight='bold')
    
    ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))
    plt.rc('xtick',labelsize=13)
    plt.rc('ytick',labelsize=13)
    plt.xticks(rotation=0)
    plt.legend(title=legend,prop={'size': 12})
    filename=var1+"_"+var2+".pdf"
    plt.savefig(filename,bbox_inches='tight')    
    

In [ ]:
plot_2_variable(df,var1='checking-account', var2='class-label', width=0.8, wide=8,height=4,
                legend="Class", xlabel="Status of checking account", logy=False,verbose=True,indexlist=["no account", "<0 DM","0 <= <200 DM",">= 200 DM "],index=True)

In [ ]:
#Credit amount
sns.distplot(df['credit-amount'], hist=True, kde=False, 
             bins=40, color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

In [ ]:
#Duration
sns.distplot(df['duration'], hist=True, kde=False, 
             bins=10, color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

# Load Dataset

## German Credit

In [ ]:
def load_german_credit():
    df = pd.read_csv('data/german_data_credit.csv')    
    protected_attribute = 'sex'
    majority_group_name = "male"
    minority_group_name = "female"
    class_label = 'class-label' 
    
    print("Length:",len(df))
    print("Number of attribute:",len(df.columns))
    
    print("Length (cleaned):",len(df))
    print("Class imbalance: \n",df[class_label].value_counts())
    #label encode
    le = preprocessing.LabelEncoder()
    for i in df.columns:
        if df[i].dtypes == 'object':
            df[i] = le.fit_transform(df[i])
    #Splitting data into train and test
    length = len(df.columns)
    X = df.iloc[:,:length-1]
    y = df[class_label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) 
    
    #Get index    
    feature = X.keys().tolist()    
    sa_index = feature.index(protected_attribute)
    p_Group = 0 
    
    return X_train, X_test, y_train, y_test,sa_index, p_Group, protected_attribute,majority_group_name,minority_group_name

# Run experiment

In [ ]:
def run_experiment(dataset, X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name, algorithm, preprocessing_algorithm='', postprocessing_algorithm=''):    
    datasetTrain = BinaryLabelDataset(df=pd.concat([X_train, y_train.to_frame()], axis=1), label_names=[y_train.name], protected_attribute_names=[protected_attribute])
    datasetTest = BinaryLabelDataset(df=pd.concat([X_test, y_test.to_frame()], axis=1), label_names=[y_test.name], protected_attribute_names=[protected_attribute])
    
    # preprocessing
    if preprocessing_algorithm != '': 
        if preprocessing_algorithm == 'DIR':
            pre_model = DisparateImpactRemover(sensitive_attribute=protected_attribute)
        elif preprocessing_algorithm == 'LFR':
            privileged_groups = [{protected_attribute: 1.0}]
            unprivileged_groups = [{protected_attribute: 0.0}]
            pre_model = LFR(unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
        dataset_train_transf = pre_model.fit_transform(datasetTrain)
        dataset_test_transf = pre_model.transform(datasetTest)

        X_train_transf = dataset_train_transf.features
        y_train_transf = dataset_train_transf.labels.ravel()
        X_test_transf = dataset_test_transf.features
        y_test_transf= dataset_test_transf.labels.ravel()

        X_train_transf = pd.DataFrame(X_train_transf, columns = X_train.columns)
        y_train_transf = pd.Series(y_train_transf, name = y_train.name).astype(int)
        X_test_transf = pd.DataFrame(X_test_transf, columns = X_train.columns)
        y_test_transf = pd.Series(y_test_transf, name = y_train.name).astype(int)
    
    # inprocessing
    if algorithm == 'DT':
        model = tree.DecisionTreeClassifier(random_state=0)  
    elif algorithm == 'NB': 
        model = GaussianNB()
    elif algorithm == 'MLP':
        model = MLPClassifier(random_state=1, max_iter=300)
    elif algorithm == 'kNN':
        model = KNeighborsClassifier(n_neighbors=5)
    elif algorithm == 'Agar':
        clf =  NB = GaussianNB()
        model = ExponentiatedGradientReduction(prot_attr=protected_attribute,estimator=clf, constraints = "EqualizedOdds")
    
    if preprocessing_algorithm != '':
        model.fit(X_train_transf, y_train_transf)
        y_predicts = model.predict(X_test_transf)
    else:
        model.fit(X_train,y_train)
        y_predicts = model.predict(X_test)

    # postprocessing
    if postprocessing_algorithm != '':
        privileged_groups = [{protected_attribute: 1.0}]
        unprivileged_groups = [{protected_attribute: 0.0}]
        if postprocessing_algorithm == 'EOP':
            post_model = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=42)
        if postprocessing_algorithm == 'CEP':
            post_model = CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=42)
        
        y_test_predicts = model.predict(X_test)
        y_train_predicts = model.predict(X_train)

        X_train_predicts = X_train.copy()
        X_test_predicts = X_test.copy()

        X_train_predicts[y_train.name] = y_train_predicts
        X_test_predicts[y_train.name] = y_test_predicts
    
        dataset_train_true = BinaryLabelDataset(df=pd.concat([X_train, y_train.to_frame()], axis=1), label_names=[y_train.name], protected_attribute_names=[protected_attribute])
        dataset_train_predicts = BinaryLabelDataset(df=X_train_predicts, label_names=[y_train.name], protected_attribute_names=[protected_attribute])
        dataset_test_predicts = BinaryLabelDataset(df=X_test_predicts, label_names=[y_test.name], protected_attribute_names=[protected_attribute])

        post_model.fit_predict(dataset_true=dataset_train_true, dataset_pred=dataset_train_predicts)
        dataset_predicts_transf = post_model.predict(dataset_test_predicts)
        data_predicts = dataset_predicts_transf.convert_to_dataframe()[0]
        y_predicts = data_predicts[y_test.name].astype(int)

    print("Statistical parity:")
    print(calculate_performance_statistical_parity(X_test.values, y_test.values, y_predicts, sa_index, p_Group))
         
    print("Equal opportunity")
    print(calculate_performance_equal_opportunity(X_test.values, y_test.values, y_predicts,  sa_index, p_Group))
        
    print("Equalized odds")
    print(calculate_performance_equalized_odds(X_test.values, y_test.values, y_predicts, sa_index, p_Group))
         
    print("Predictive parity")
    print(calculate_performance_predictive_parity(X_test.values, y_test.values, y_predicts,  sa_index, p_Group))
        
    print("Predictive equality")
    print(calculate_performance_predictive_equality(X_test.values, y_test.values, y_predicts,  sa_index, p_Group))
        
    print("Treatment equality")
    print(calculate_performance_treatment_equality(X_test.values, y_test.values, y_predicts,  sa_index, p_Group))
        
    filename = '{}.{}.abroca.pdf'.format(dataset, preprocessing_algorithm+algorithm)
    #make predictions
    if postprocessing_algorithm == '':
        if preprocessing_algorithm:
            X_test['pred_proba'] = model.predict_proba(X_test_transf)[:,1:2]
        else:
            X_test['pred_proba'] = model.predict_proba(X_test)[:,1:2]
        X_test['true_label'] = y_test
        df_test = X_test

        #Compute Abroca
        slice = compute_abroca(df_test, pred_col = 'pred_proba' , label_col = 'true_label', protected_attr_col = protected_attribute,
                            majority_protected_attr_val = 1, n_grid = 10000,
                            plot_slices = True, majority_group_name=majority_group_name ,minority_group_name=minority_group_name,file_name = filename)
        print("ABROCA:",slice)
        plt.clf() 
    plt.clf() 
    

In [ ]:
#Main function
def run_eval(dataset, algorithm, preprocessing_algorithm='', postprocessing_algorithm=''):
    if dataset == 'credit-approval':
        X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name = load_credit_approval()
        run_experiment(dataset, X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name, algorithm, preprocessing_algorithm, postprocessing_algorithm)                                        
    if dataset == 'credit-card':
        X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name = load_credit_card()
        run_experiment(dataset, X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name, algorithm, preprocessing_algorithm, postprocessing_algorithm)                                        
    if dataset == 'german-credit':
        X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name = load_german_credit()
        run_experiment(dataset, X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name, algorithm, preprocessing_algorithm, postprocessing_algorithm)                                            
    if dataset == 'PAKDD':
        X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name = load_PAKDD2010()
        run_experiment(dataset, X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name, algorithm, preprocessing_algorithm, postprocessing_algorithm)                                                                
    if dataset == 'credit-scoring':
        X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name = load_credit_scoring()
        run_experiment(dataset, X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name, algorithm, preprocessing_algorithm, postprocessing_algorithm)                                                                        
    if dataset == 'application':
        X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name = load_application()
        run_experiment(dataset, X_train, X_test, y_train, y_test,sa_index, p_Group,protected_attribute,majority_group_name,minority_group_name, algorithm, preprocessing_algorithm, postprocessing_algorithm)                                                                       

## German Credit


In [ ]:
run_eval(dataset='german-credit', algorithm='DT')

In [ ]:
run_eval(dataset='german-credit', algorithm='NB')

In [ ]:
run_eval(dataset='german-credit', algorithm='MLP')

In [ ]:
run_eval(dataset='german-credit', algorithm='kNN')

In [ ]:
run_eval(dataset='german-credit', algorithm='Agar')

In [ ]:
run_eval(dataset='german-credit', algorithm='DT', preprocessing_algorithm='DIR')

In [ ]:
run_eval(dataset='german-credit', algorithm='NB', preprocessing_algorithm='DIR')

In [ ]:
run_eval(dataset='german-credit', algorithm='MLP', preprocessing_algorithm='DIR')

In [ ]:
run_eval(dataset='german-credit', algorithm='kNN', preprocessing_algorithm='DIR')

In [ ]:
run_eval(dataset='german-credit', algorithm='DT', preprocessing_algorithm='LFR')

In [ ]:
run_eval(dataset='german-credit', algorithm='NB', preprocessing_algorithm='LFR')

In [ ]:
run_eval(dataset='german-credit', algorithm='MLP', preprocessing_algorithm='LFR')

In [ ]:
run_eval(dataset='german-credit', algorithm='kNN', preprocessing_algorithm='LFR')

In [ ]:
run_eval(dataset='german-credit', algorithm='DT', postprocessing_algorithm='EOP')

In [ ]:
run_eval(dataset='german-credit', algorithm='NB', postprocessing_algorithm='EOP')

In [ ]:
run_eval(dataset='german-credit', algorithm='MLP', postprocessing_algorithm='EOP')

In [ ]:
run_eval(dataset='german-credit', algorithm='kNN', postprocessing_algorithm='EOP')

In [ ]:
run_eval(dataset='german-credit', algorithm='DT', postprocessing_algorithm='CEP')

In [ ]:
run_eval(dataset='german-credit', algorithm='NB', postprocessing_algorithm='CEP')

In [ ]:
run_eval(dataset='german-credit', algorithm='MLP', postprocessing_algorithm='CEP')

In [ ]:
run_eval(dataset='german-credit', algorithm='kNN', postprocessing_algorithm='CEP')